In [1]:
from tqdm import tqdm
import pickle as pkl
import pandas as pd
import re

In [2]:
def clean(text):
    return re.sub(r'[^a-zA-Z ]', "", text).lower().lstrip().rstrip()

In [9]:
def remove_stop_words(np):
    
    stop_words = [line.rstrip('\n') for line in open('stopwordList.txt')]
    new_np = ''
    words = list(np.split()) 
    words = [w for w in words if w.lower() not in stop_words]
    word_to_add = ' '.join(words)
    if len(word_to_add) > 1:
        new_np = word_to_add
    return new_np

In [5]:
ne_np_sent_dicts = pkl.load(open('temp/ne_np_sent_all.pkl', 'rb'))

In [6]:
print(ne_np_sent_dicts[100])
print('11'.isnumeric())

{'ne_np': ['evening'], 'sentence': 'As always the evening was wonderful and inspiring to watch, and I am very excited to have a good one.'}
True


In [7]:
def list_of_dict_text(ne_np_sent_dicts):
    
    stop = [line.rstrip('\n') for line in open('stop.txt')]
#     txt = [{'ne_np': ['a  ad bubble exclusion zones', 'a  billion bailout', 'a  de la grande guerre patriotique', 'a  fillup', 'a  job', 'a  million antiterrorist center', 'a  trillion tax cut', 'a  year old asian woman', 'a a'], 'sentence': "Someone had to draw my att"}]
    ne_np_sent_clean = []
    for ne_np_sent in tqdm(ne_np_sent_dicts):
        clean_ne_np = []
        for ne in ne_np_sent['ne_np']:
            ne = clean(ne)
            ne = remove_stop_words(ne)
    #     ne = ''
    #     try:
    #         ne = clean(item[0])
    #         print(ne)
    #     except:
    #         pass
            if ne != '' and len(ne) > 2 and ne.isnumeric() == False and ne.lower() not in stop:
                clean_ne_np.append(ne)
        if len(clean_ne_np) > 0:    
            ne_np_sent_clean.append({'ne_np':clean_ne_np, 'sentence': ne_np_sent['sentence']})
    return ne_np_sent_clean

In [10]:
ne_np_sent_clean = list_of_dict_text(ne_np_sent_dicts)
pkl.dump(ne_np_sent_clean, open('temp/ne_np_sent_clean.pkl', 'wb'))

100%|███████████████████████████████████████████████████████████████████████████| 41941/41941 [00:49<00:00, 841.67it/s]


In [11]:
import pickle as pkl
ne_np_sent_clean_final = []
for item in tqdm(ne_np_sent_clean):
    tmp = list(set(item['ne_np']))
    ne_np_sent_clean_final.append({'ne_np': tmp, 'sentence': item['sentence']})

print(len(ne_np_sent_clean_final))
pkl.dump(ne_np_sent_clean_final, open('temp/ne_np_sent_clean_final.pkl', 'wb'))
print(ne_np_sent_clean_final[0])

100%|████████████████████████████████████████████████████████████████████████| 38715/38715 [00:00<00:00, 289216.23it/s]

38715
{'ne_np': ['sales', 'theesehese', 'advantheeseage'], 'sentence': 'Take advantheeseage of theesehese One, theeserembling sales!'}


In [12]:
tmp = pkl.load(open('temp/ne_np_sent_clean_final.pkl', 'rb'))
tmp = list_of_dict_text(tmp)

100%|███████████████████████████████████████████████████████████████████████████| 38715/38715 [00:53<00:00, 719.03it/s]


In [13]:
def get_features_vocab(input_list):
    # input to this function is a list of dicts
    #  sample of input:  [{'ne_np':['number', 'letters', 'daily mail', 'sentence': "Someone had to draw my attention to these 2 letters published by the Daily Mail"]},
    #                     {'ne_np':['adc','dxc'], 'sentence': "llgh sleihg sglh"}]
    features = []
    vocab = []
    for item in tqdm(input_list):
        features.append(item['ne_np'])
        vocab.extend(item['ne_np'])
    # features is a list of list
    # vocab is list of str,
    return features, sorted(list(set(vocab)))


In [14]:
features , vocab = get_features_vocab(tmp)

100%|███████████████████████████████████████████████████████████████████████| 38715/38715 [00:00<00:00, 2193557.48it/s]


In [15]:
print(vocab[:200])

['aaaaaaaa', 'aaaaaaaaagh', 'aaaaaaaahhhh', 'aaaaaarrrrrrrrr', 'aaaand', 'aaah', 'aabound', 'aah', 'aahs', 'aannndd', 'aannndd afteralll', 'aanythingonigone', 'aanythingonigone lord production', 'aaron', 'abandon bathroom', 'abanyone', 'abboutbout', 'abboutbout nucleabboutr power', 'abboutnd', 'ability', 'abis', 'ablout', 'abodes', 'aboion', 'abortion', 'abotu bottle', 'abotubout', 'abotubout gabotus abotund food', 'abotund', 'aboutcan', 'abouteh tehhe sewers untehil justeh', 'aboutheese', 'abouthge', 'aboutje', 'aboutje tjehe foliukeliukeowers', 'aboutwonder', 'abrupt politically expedient someneomeone', 'abs', 'absolute champ', 'absolute savage coyote roast', 'absolute terror', 'absolute trust', 'absolutehely loved iteh', 'absolutely', 'absolutely flavor', 'absolutely horrible reception', 'abstinence', 'abstract modern art section', 'absurd', 'abuse', 'academics', 'academy', 'academy training room', 'acaulco', 'acc', 'accenhtey', 'accent', 'accents', 'access', 'accessory', 'accidantc

In [16]:
vocab = []
for i in range(len(features)):
    vocab.extend(features[i])
vocab = sorted(list(set(vocab)))
print(len(vocab))

16853


In [17]:
counter = {}

In [18]:
for ne_np_sent_dict in tqdm(ne_np_sent_dicts):
    for ne_np in ne_np_sent_dict['ne_np']:
        counter[ne_np] = 0

100%|███████████████████████████████████████████████████████████████████████| 41941/41941 [00:00<00:00, 1105976.50it/s]


In [19]:
len(counter)

print(list(counter.keys())[:10])

['theesehese', 'advantheeseage', 'sales', 'someneomeone', 'behinderte', 'mieten', 'und behinderte', 'month', 'vibedorerating opportunity', 'opportunity']


In [20]:
for ne_np_sent_dict in tqdm(ne_np_sent_dicts):
    for ne_np in ne_np_sent_dict['ne_np']:
        counter[ne_np] += 1

100%|████████████████████████████████████████████████████████████████████████| 41941/41941 [00:00<00:00, 982673.54it/s]


In [21]:
print(counter['Daily Mail'])

KeyError: 'Daily Mail'

In [22]:
final = []
for key, value in tqdm(counter.items()):
    final.append([key, value])

100%|███████████████████████████████████████████████████████████████████████| 17123/17123 [00:00<00:00, 1574668.76it/s]


In [23]:
ne_np_df = pd.DataFrame(final, columns = ['ne_np', 'count'])


In [24]:
print(len(ne_np_df.values.tolist()))

17123


In [25]:
stop = [line.rstrip('\n') for line in open('stop.txt')]
values = {}
for item in ne_np_df.values.tolist():
    ne = clean(item[0])
#     ne = ''
#     try:
#         ne = clean(item[0])
#         print(ne)
#     except:
#         pass
    
    if ne != '' and len(ne) > 2 and ne.lower() not in stop:
            if ne in values:
                values[ne] += item[1]
            else:
                values[ne] = item[1]
        

In [26]:
print(len(values))

16987


In [27]:
final_values = []
for key, item in values.items():
    final_values.append([key, item])
#final_frame.to_csv('ne_np_features.csv', index = False)

In [28]:
final_frame = pd.DataFrame(final_values, columns = ['ne_np' , 'count'])
final_frame = final_frame.loc[~final_frame['ne_np'].str.isnumeric()]
filter_frame = final_frame.loc[final_frame['count'] >= 50]
filter_frame = filter_frame.sort_values(by='count', ascending = False)
print(filter_frame)
filter_frame.to_csv('new_YInt_NE_NP.csv', index = False)

          ne_np  count
143      people   1584
85        power   1350
1339      water   1110
1336       city   1037
405        food    928
...         ...    ...
5590      smoke     50
12821   grandpa     50
4201   drinking     50
2836      north     50
3344       pics     50

[547 rows x 2 columns]


In [29]:
final_frame = final_frame.sort_values(by='count', ascending = False)
print(final_frame)
final_frame.to_csv('new_YInt_NE_NP_unfiltered.csv', index = False)

                     ne_np  count
143                 people   1584
85                   power   1350
1339                 water   1110
1336                  city   1037
405                   food    928
...                    ...    ...
8590           objectively      1
8591                 magic      1
8595   doriscurioushoggard      1
8596                spothe      1
16986               morals      1

[16987 rows x 2 columns]
